# Stitching thermal drone imagery

In [6]:
# Variable definition
imgDir = 'data/lsm/Flight 1 - Library/'
path = 'data/lsm/Flight 1.csv'

In [3]:
#Import packages
%pip install opencv-python
import glob
import rasterio as ro
import cv2 as cv

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Retrieve individual images
imgs = [_ for _ in glob.glob(imgDir+'*.*') if _.endswith(('.jpg','.png','.tif'))]

In [4]:
"""Raster import functions"""
# Convert image raster to array
def raster_to_array(file, crop=False):
    img = ro.open(file)
    return img.read()
    #return get_training_array(img) if crop else img.read()

# Function taking a raster input and outputting pandas dataframe
def raster_to_df(file, cn='class', multidims = False, crop = False):
    arr = raster_to_array(file, crop)
    if multidims:
        df = pd.DataFrame(arr.reshape(-1,len(arr)))
        df['x'], df['y'] = np.tile(np.arange(arr.shape[2]),arr.shape[1]), np.repeat(range(arr.shape[1]),arr.shape[2])
        df = df.set_index(['y','x'])
    else: df = pd.DataFrame(arr[0]).stack().rename_axis(['y', 'x']).reset_index(name=cn).set_index(['y','x'])
    return df, arr

## Rasterio read and warp

In [8]:
img = ro.open(imgs[0])

/opt/conda/lib/python3.7/site-packages/rasterio/__init__.py:216: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [10]:
img.

array([[[232, 229, 232, ..., 176, 173, 171],
        [244, 235, 233, ..., 176, 174, 173],
        [236, 234, 235, ..., 173, 173, 173],
        ...,
        [231, 231, 232, ..., 142, 134, 130],
        [227, 230, 233, ..., 138, 134, 131],
        [227, 231, 234, ..., 137, 134, 132]],

       [[108, 105, 108, ...,  39,  40,  41],
        [121, 113, 111, ...,  39,  40,  41],
        [116, 114, 116, ...,  39,  39,  39],
        ...,
        [ 60,  60,  61, ...,  49,  50,  53],
        [ 56,  59,  61, ...,  48,  50,  52],
        [ 56,  60,  62, ...,  48,  49,  51]],

       [[ 10,   5,   8, ...,  67,  67,  67],
        [ 25,  14,  12, ...,  67,  67,  65],
        [ 21,  17,  16, ...,  66,  66,  64],
        ...,
        [ 42,  43,  44, ...,  94, 100, 105],
        [ 36,  42,  47, ..., 100, 102, 107],
        [ 36,  43,  48, ..., 106, 106, 107]]], dtype=uint8)

## OpenCV stitching

In [5]:
img = cv.imread(imgs[0])
cv.imshow("Display",img)